In [139]:
from qpth.qp import QPFunction
import torch
import numpy as np
from torch.autograd import Variable

In [140]:
nz = 10
nineq = 25
neq = 10

In [141]:
# script to create and save non random trivial QP

import numpy.random as npr

Q = npr.rand(nz, nz)
Q = np.matmul(Q, Q.T)
G = npr.randn(nineq, nz)
z0 = npr.randn(nz)
s0 = npr.rand(nineq)
q = npr.randn(nz)
h = np.matmul(G, z0) + s0
A = npr.randn(neq, nz)
b = np.matmul(A,z0)

names = ["Q", "q", "G", "h", "A", "b"]
matrices = [Q, q, G, h, A, b]

for i in range(len(names)):
    mat = np.matrix(matrices[i])
    with open(names[i]+'.txt','wb') as fi:
        for line in mat:
            np.savetxt(fi, line, fmt='%.9f')

In [142]:
# read matrices from files
names = ["Q", "q", "G", "h", "A", "b"]
matrices = []

for name in names:
    with open(name+'.txt','rb') as fi:
        matrices.append(np.loadtxt(name+'.txt'))
    
Q, q, G, h, A, b = matrices

In [143]:
q, Q, G, h, A, b = [torch.Tensor(x) for x in [q, Q, G, h, A, b]]

q, Q, G, h, A, b = [Variable(x) for x in [q, Q, G, h, A, b]]
Q.requires_grad = True
q.requires_grad = True
G.requires_grad = True
h.requires_grad = True
A.requires_grad = True
b.requires_grad = True

In [144]:
qpf = QPFunction()
z_hat = qpf(Q, q, G, h, A, b)

In [145]:
z = z_hat.detach().numpy()[0]
z # solution

array([-1.501465  , -1.4053086 ,  0.59770143, -0.23384869,  0.8757396 ,
       -0.14110026,  0.04140254, -0.9955775 ,  0.7384028 ,  0.5887077 ],
      dtype=float32)

In [146]:
z_hat.backward(torch.ones(1, nz))

In [147]:
grads = [x.grad.data.squeeze(0).cpu().numpy() for x in [Q, q, G, h, A, b]]

In [154]:
# saving gradients to file
names = ["dQ", "dq", "dG", "dh", "dA", "db"]

for i in range(len(names)):
    mat = np.matrix(grads[i])
    with open(names[i]+'.txt','wb') as fi:
        for line in mat:
            np.savetxt(fi, line, fmt='%.9f')